# Roundabout

In [ ]:
from math import pi

import numpy as np

from pyspect import *
from pyspect.langs.ltl import *

TLT.select(ContinuousLTL)

## Environment

In [ ]:
## CONSTANTS ##

LONVEL = (0.3, 0.6) # [m/s]
LATVEL = (0, 0.2)   # [m/s]

RBOUND = (0.8, 1.6) # [m]

RI = 1.0    # [m]
RM = 1.3    # [m]
RO = 1.6    # [m]

PHI_LOOKAHEAD = 2 # [rad]
TIME_HORIZON = 2 # [s]; comp time horizon

## TRAFFIC RULES ##

lonspeed = BoundedSet(v_phi=LONVEL)
latspeed = BoundedSet(v_r=LATVEL)

## ROUNDABOUT ##

lanes = And(BoundedSet(r=(RI, RO)), lonspeed, latspeed)

inner = And(BoundedSet(r=(RI, RM)), lanes)
outer = And(BoundedSet(r=(RM, RO)), lanes)

## TASKS ##

stay_inner = Until(inner, inner)
stay_outer = Until(outer, outer)
goto_inner = Until(lanes, inner)
goto_outer = Until(lanes, outer)

## Implementations

In [ ]:
IMPL = 'HJ'

if IMPL == 'HJ':

    import hj_reachability as hj
    from pyspect.impls.hj_reachability import TVHJImpl
    from hj_reachability.systems import Bicycle5DCircular

    dynamics = dict(cls=Bicycle5DCircular,
                    min_steer=-pi/4, 
                    max_steer=+pi/4,
                    min_accel=-0.4,
                    max_accel=+0.4)

    #                        r,                  phi,  v_r,  v_phi,   v_yaw
    max_bounds = np.array([1.8,       +PHI_LOOKAHEAD, +0.1,  +0.7,   +pi/5])
    min_bounds = np.array([0.8, 0.5 * -PHI_LOOKAHEAD, -0.1,  +0.3,   -pi/5])
    grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(hj.sets.Box(min_bounds, max_bounds),
                                                                (11, 51, 5, 5, 5))

    impl = TVHJImpl(dynamics, grid, TIME_HORIZON)
    impl.set_axes_names('t', 'r', 'phi', 'v_r', 'v_phi', 'v_yaw')

if IMPL == 'HZ':
        
    from hz_reachability.hz_impl import HZImpl
    from hz_reachability.systems.cars import CircularBicycle5DLinearized

    z0 = []
    u0 = []
    dynamics = CircularBicycle5DLinearized(z0, u0)
    # space = 
    time_horizon = TIME_HORIZON
    time_step = 0.2

    impl = HZImpl(dynamics, space, time_horizon, time_step, show_intermediate=True)


In [ ]:
from pyspect.plotting.levelsets import *

# out, s = TLT(goto_outer).realize(impl)
# print(s)

terminal = Or(BoundedSet(t=(1.9, 2.1)),
              And(lanes, BoundedSet(phi=(1.7, 2.1))))

# Static obstacle
OBS1 = BoundedSet(r=(RI, RM), phi=(1.0, 1.3))

# Dynamic obstacle - reachF not supported by LTL so we use special hacks
obs2 = BoundedSet(t=(0.5, 1.5), r=(RM, RO), phi=(0.3, 0.6))(impl)
OBS2 = TLT(Set(
    impl.project_onto(
        impl.reachF(obs2, outer.realize(impl)),
        0, 1, 2,
        keepdims=True,
    )
))

# task = Until(goto_outer, terminal)
# task = Until(And(Not(OBS1), goto_outer), terminal)
task = Until(And(Not(OBS1), Not(OBS2), goto_outer), terminal)

tmp = OBS1(impl)
tmp = impl.make_tube(tmp)

out = task.realize(impl)
out = impl.project_onto(out, 0, 1, 2)

In [ ]:

# plot_levelsets(
#     (impl.project_onto(OBS2.realize(impl), 0, 1, 2, union=False), {"colorscale": "reds"}),
#     (impl.project_onto(tmp, 0, 1, 2, union=False), {"colorscale": "reds"}),
#     # # (impl.project_onto(goto_outer.realize(impl), 0, 1, 2), {"colorscale": "greys"}),
#     out,
#     plot_func=plot3D_levelset, fig_theme='Light',
#     min_bounds=[impl.timeline[ 0], *min_bounds[:2]],
#     max_bounds=[impl.timeline[-1], *max_bounds[:2]],
#     xtitle="x [m]", ytitle="y [m]",
#     fig_width=500, fig_height=500,
#     eye=EYE_MH_S,
# ).show()

plot2D_levelset(
    # out[5],
    impl.project_onto(impl.plane_cut(offset=[1.2], normal=[1.0], axes=[1]), 1, 2),
    min_bounds=min_bounds[:2],
    max_bounds=max_bounds[:2],
    xtitle="x [m]", ytitle="y [m]",
    fig_width=500, fig_height=500,

).show()

plot3D_valuefun(
    out[5],
    min_bounds=min_bounds[:2],
    max_bounds=max_bounds[:2],
    xtitle="x [m]", ytitle="y [m]",
    fig_width=500, fig_height=500,
).show()